<a href="https://colab.research.google.com/github/yilinearity/Kinetic_Fit_Simulation/blob/main/BiMol_Irrev_FitAB_v1_3_GColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fitting Kinetic Data to a + b > c (Irreversible)**

```
d[a]/dt = -k * [a] * [b]
d[b]/dt = -k * [a] * [b]
rate constant = k
```

The input data should be in a csv format, with three columns **Time, [A], and [B]**. The presence of the fourth column **[C]** is optional. The first row is the heading row.

(Another Jupyter Notebook 'BiMol_Irrev_FitABC_v1.3.ipynb' will make use of [C] for fitting.)

## **To Use**
The script has two cells; <font color='blue'>click on the **run** icon (on the left side of the code) to run each cell.</font>

In the first cell, the data csv file will be first imported (uploaded), and a slider will show up to select a good initial value of ***k_initial***. Note that this ***k_initial*** is the exponent of ***k*** (i.e. *k* = 10^(***k_initial***)). The current range is set between (-10.0, 3.0); change it to suit your need. A good initial value for ***k*** will ensure the successful fit of the data set.

The value of ***k_initial*** is then used in the second cell to complete the fitting routine, which made use of `odeint` from `scipy.integrate`. Finally, the results are saved and exported as a csv (in Google drive); the rate constant ***k*** will be displayed on the screen. Modify the path of the Google Drive folder if needed. (Be patient, the saving takes time on Google Drive.)

<br>

---
MIT License Copyright (c) 2023 Yi-Lin Wu (yilinearity@gmail.com) <br>
Full text of the MIT License can be found at: https://opensource.org/licenses/MIT

In [ ]:
import os
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from ipywidgets import interactive

# Define the system of ODEs
def system(y, t, k):
    a, b = y
    dadt = -k * a * b
    dbdt = -k * a * b
    return [dadt, dbdt]

# Function to upload and process the CSV file
def process_uploaded_file(uploaded_file):
    data = pd.read_csv(uploaded_file)
    t = data.iloc[:, 0]
    a = data.iloc[:, 1]
    b = data.iloc[:, 2]
    return t, a, b

# Initialize the k variable as a global variable
k = 10**(-7)

def plot(k_initial=-7):
    global k  # Declare k as a global variable
    k = 10**k_initial  # Update the global k value

    # Check if the data is available
    if 't' not in globals() or 'a' not in globals() or 'b' not in globals():
        print("Please upload a CSV file first.")
        return

    # Solve the system of ODEs with the global k value
    sol = odeint(system, [a[0], b[0]], t, args=(k,))
    a_fit = sol[:, 0]
    b_fit = sol[:, 1]

    # Plot the results
    plt.plot(t, a, 'o', label='a (data)')
    plt.plot(t, b, 'o', label='b (data)')
    plt.plot(t, a_fit, label='a (fit)')
    plt.plot(t, b_fit, label='b (fit)')
    plt.legend()
    plt.show()

    return k_initial

# Use Colab's file upload feature to upload the CSV file
from google.colab import files
uploaded = files.upload()

# Process the uploaded file
for filename in uploaded.keys():
    t, a, b = process_uploaded_file(filename)

# Create the interactive plot widget
w = interactive(plot, k_initial=(-10.0, 3.0))
w


In [ ]:
from google.colab import drive

k_exp = w.kwargs['k_initial']

print(f"The initial value of k is: {10**k_exp:.2f}")

def fit_k(x, k):
    sol = odeint(system, [a[0], b[0]], t, args=(k,))
    return (sol[:, 0] - a)**2 + (sol[:, 1] - b)**2

k_initial = 10**k_exp
popt, pcov = curve_fit(fit_k, xdata=None, ydata=np.zeros_like(t), p0=[k_initial], method='lm')
k = popt[0]
perr = np.sqrt(np.diag(pcov))

sol = odeint(system, [a[0], b[0]], t, args=(k,))
a_fit = sol[:, 0]
b_fit = sol[:, 1]

plt.plot(t, a, 'o', label='a (data)')
plt.plot(t, b, 'o', label='b (data)')
plt.plot(t, a_fit, label='a (fit)')
plt.plot(t, b_fit, label='b (fit)')
plt.legend()
plt.show()

# Display the results
print(f"k = {k:.4e} ± {perr[0]:.4e}")

output = pd.DataFrame({'t': t, 'a': a, 'b': b, 'a_fit': a_fit, 'b_fit': b_fit})
print(output)

# Save the DataFrame to a CSV file in Google Drive
drive.mount('/content/drive')
output.to_csv('/content/drive/My Drive/results/BiMol_Irrev_Fit_AB.csv', sep=",", index=False)
